In [33]:
### imports

# external modules
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import SubHistStruct
importlib.reload(SubHistStruct)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

<module 'HyperRectangleFitter' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/cloudfitters/HyperRectangleFitter.py'>

In [2]:
### Speed Controls and Run Mode

# Disables all plots for large datasets where speed is more important
createPlots = False

# Control for the notebook - turn off user-friendly mode to enable faster runtimes
userfriendly = True

In [3]:
### Evaluation Parameters

# Select the bias towards recall against precision, treated as a factor (so < 1 biases towards precision, 1 is equal importance, and > 1 biases towards recall)
biasFactor = 20

In [15]:
### Data Controls and Selection - 1D Autoncoder

# The directory data is located in
datadir = '../data/'

# Select a list of good runs to train on in development training_mode
# Should be validated by eye
trainrunsls = {'2017B':{
#                    "297057":[[-1]], 
#                    "297099":[[-1]], 
#                    "297101":[[-1]],
#                    "297113":[[-1]], 
#                    "297114":[[-1]], 
                    "297175":[[-1]],   # A decently clean histogram
#                    "297177":[[-1]],
#                    "297179":[[-1]], 
#                    "297215":[[-1]],
#                    "297218":[[-1]],
#                    "297225":[[-1]],
#                    "297296":[[-1]], 
#                    "297411":[[-1]],
#                    "297426":[[-1]],  
#                    "297431":[[-1]],
#                    "297434":[[-1]], 
#                    "297468":[[-1]],
#                    "297483":[[-1]],
#                    "297486":[[-1]],
#                    "297503":[[-1]],
#                    "297557":[[-1]],
#                   "297598":[[-1]],
#                   "297604":[[-1]],   # A decently clean histogram
                   "297620":[[-1]],   # A decently clean histogram
                   "297659":[[-1]],   # An okay histogram
                   "297670":[[-1]],   # A decently clean histogram
#                    "297674":[[-1]],
#                    "297678":[[-1]],   # A particularly messy histogram
#                    "297722":[[-1]],   # A decently clean histogram
#                    "298997":[[-1]],
#                    "299061":[[-1]],
                   "299065":[[-1]],   # A decently clean histogram
                   "299067":[[-1]],   # A decently clean histogram
                   "299096":[[-1]],
                   "299149":[[-1]],
#                    "299178":[[-1]],   # A decently clean histogram
#                    "299184":[[-1]],   # A particularly messy histogram
#                    "299185":[[-1]],   # A decently clean histogram
#                    "299327":[[-1]],
#                    "299329":[[-1]], 
#                    "299480":[[-1]]    # A decently clean histogram
                    },
                   '2017C': {
                      "299370":[[-1]],
                      "299394":[[-1]],
                      "299420":[[-1]],
                      "299477":[[-1]],
                      "299593":[[-1]],
                      "299597":[[-1]],
                      "299617":[[-1]],
                      "300018":[[-1]],
                      "300105":[[-1]],
                      "300117":[[-1]],
                      "300124":[[-1]],
                      "300234":[[-1]],
                      "300237":[[-1]],
                      "300240":[[-1]],
                      "300370":[[-1]],
                      "300157":[[-1]],
                      "300373":[[-1]],
                      "300392":[[-1]],
                      "300395":[[-1]],
                      "300401":[[-1]],
                      "300462":[[-1]],
                      "300466":[[-1]],
                      "300514":[[-1]],
                      "300517":[[-1]],
                      "300538":[[-1]],
                      "300539":[[-1]],
                      "300364":[[-1]],
                 },'2017F':{
                      "305310":[[-1]],
                      "305040":[[-1]],
                      "305043":[[-1]],
                      "305185":[[-1]],
                      "305204":[[-1]],
                      "305234":[[-1]],
                      "305247":[[-1]],
                      "305313":[[-1]],
                      "305338":[[-1]],
                      "305350":[[-1]],
                      "305364":[[-1]],
                      "305376":[[-1]],
                      "306042":[[-1]],
                      "306051":[[-1]],
                      "305406":[[-1]],
                      "306122":[[-1]],
                      "306134":[[-1]],
                      "306137":[[-1]],
                      "306154":[[-1]],
                      "306170":[[-1]],
                      "306417":[[-1]],
                      "306432":[[-1]],
                      "306456":[[-1]],
                      "305516":[[-1]],
                      "305586":[[-1]],
                      "305588":[[-1]],
                      "305590":[[-1]],
                      "305809":[[-1]],
                      "305832":[[-1]],
                      "305840":[[-1]],
                      "305898":[[-1]],
                      "306029":[[-1]],
                      "306037":[[-1]],
                      "306095":[[-1]],
                },
                '2018':{ # needs to be re-checked, not guaranteed to be fully correct or representative.   
                  "315267":[[-1]] 
              }
}

# Select a list of good runs to test on in development training_mode
# Should be validated by eye
goodrunsls = {'2017B':{
#                    "297057":[[-1]], 
#                    "297099":[[-1]], 
#                    "297101":[[-1]],
#                    "297113":[[-1]], 
#                    "297114":[[-1]], 
                    "297175":[[-1]],   # A decently clean histogram
#                    "297177":[[-1]],
#                    "297179":[[-1]], 
#                    "297215":[[-1]],
#                    "297218":[[-1]],
#                    "297225":[[-1]],
#                    "297296":[[-1]], 
#                    "297411":[[-1]],
#                    "297426":[[-1]],  
#                    "297431":[[-1]],
#                    "297434":[[-1]], 
#                    "297468":[[-1]],
#                    "297483":[[-1]],
#                    "297486":[[-1]],
#                    "297503":[[-1]],
#                    "297557":[[-1]],
#                    "297598":[[-1]],
#                    "297604":[[-1]],   # A decently clean histogram
#                    "297620":[[-1]],   # A decently clean histogram
                    "297659":[[-1]],   # An okay histogram
                    "297670":[[-1]],   # A decently clean histogram
                    "297674":[[-1]],
#                    "297678":[[-1]],   # A particularly messy histogram
                    "297722":[[-1]],   # A decently clean histogram
#                    "298997":[[-1]],
#                    "299061":[[-1]],
                    "299065":[[-1]],   # A decently clean histogram
                    "299067":[[-1]],   # A decently clean histogram
#                    "299096":[[-1]],
#                    "299149":[[-1]],
#                    "299178":[[-1]],   # A decently clean histogram
#                    "299184":[[-1]],   # A particularly messy histogram
                    "299185":[[-1]],   # A decently clean histogram
                    "299327":[[-1]],
#                    "299329":[[-1]], 
                    "299480":[[-1]]    # A decently clean histogram
                    },
                '2017C':{
                      "299370":[[-1]],
                      "299394":[[-1]],
                      "299420":[[-1]],
                      "299477":[[-1]],
                      "299593":[[-1]],
                      "299597":[[-1]],
                      "299617":[[-1]],
                      "300018":[[-1]],
                      "300105":[[-1]],
                      "300117":[[-1]],
                      "300124":[[-1]],
                      "300234":[[-1]],
                      "300237":[[-1]],
                      "300240":[[-1]],
                      "300370":[[-1]],
                      "300157":[[-1]],
                      "300373":[[-1]],
                      "300392":[[-1]],
                      "300395":[[-1]],
                      "300401":[[-1]],
                      "300462":[[-1]],
                      "300466":[[-1]],
                      "300514":[[-1]],
                      "300517":[[-1]],
                      "300538":[[-1]],
                      "300539":[[-1]],
                      "300364":[[-1]],
                },'2017F':{
                      "305310":[[-1]],
                      "305040":[[-1]],
                      "305043":[[-1]],
                      "305185":[[-1]],
                      "305204":[[-1]],
                      "305234":[[-1]],
                      "305247":[[-1]],
                      "305313":[[-1]],
                      "305338":[[-1]],
                      "305350":[[-1]],
                      "305364":[[-1]],
                      "305376":[[-1]],
                      "306042":[[-1]],
                      "306051":[[-1]],
                      "305406":[[-1]],
                      "306122":[[-1]],
                      "306134":[[-1]],
                      "306137":[[-1]],
                      "306154":[[-1]],
                      "306170":[[-1]],
                      "306417":[[-1]],
                      "306432":[[-1]],
                      "306456":[[-1]],
                      "305516":[[-1]],
                      "305586":[[-1]],
                      "305588":[[-1]],
                      "305590":[[-1]],
                      "305809":[[-1]],
                      "305832":[[-1]],
                      "305840":[[-1]],
                      "305898":[[-1]],
                      "306029":[[-1]],
                      "306037":[[-1]],
                      "306095":[[-1]],
                },
                '2018':{ # needs to be re-checked, not guaranteed to be fully correct or representative.   
                  "315267":[[-1]] 
              }
}


badrunsls = {'2017B':
                {
                    #"297048":[[-1]],
                    #"297282":[[-1]],
                    #"297283":[[-1]],
                    #"297284":[[-1]],
                    #"297287":[[-1]],
                    #"297288":[[-1]],
                    #"297289":[[-1]],
                    "299316":[[-1]],
                    "299317":[[-1]],
                    "299318":[[-1]],
                    "299324":[[-1]],
                    "299326":[[-1]],
                    #"301086":[[88,126]],
                    #"301086":[[89,89]],
                    #"303948":[[1710,1710]],
                    "297047":[[-1]], #close but, true bad for all 8
                    "297169":[[-1]], #true bad for all 8
#                   "297211":[[-1]], #Reconstructs well
#                   "299325":[[-1]], #Reconstructs well
                    "297664":[[-1]], #true bad for all 8
                    "299317":[[-1]], #true bad for all 8
                    "297169":[[-1]], #true bad for all 8
#                   "297502":[[-1]]
                },
             '2017C':{
#                 "300781":[[-1]], # bad for tracking (pixels were excluded.
                 "300079":[[-1]], # is bad for strips and then also for tracking
#                 "302029":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good 
#                 "300576":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
#                 "300574":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
#                 "300282":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
#                 "301912":[[-1]], # Half bad for pixels (lost HV or readout card)  
#                 "301086":[[-1]], # Half bad for pixels (lost HV or readout card)  
#                 "300283":[[-1]], # Half bad for pixels (lost HV or readout card) 
                 "300282":[[-1]], # Half bad for pixels (lost HV or readout card) 
#                 "300281":[[-1]], # Half bad for pixels (lost HV or readout card) 
#                 "300239":[[-1]], # Half bad for pixels (lost HV or readout card)
#                 "301394":[[-1]], # Marginal for pixels
#                 "301183":[[-1]], # Marginal for pixels
                 "300398":[[-1]], # Marginal for pixels
                 "300389":[[-1]], # Marginal for pixels
#                 "300365":[[-1]]  # Marginal for pixels
              },
             '2017E':{
                 "304740":[[-1]], # Bad for pixels and tracking - holes in PXLayer 1
                 "304776":[[-1]], # Bad for pixels and tracking - holes in PXLayer 1
                 "304506":[[-1]], # Portcard problem for pixels
                 "304507":[[-1]], # Portcard problem for pixels 
                 "303989":[[-1]], # Bad for pixels, power supply died
                 "303824":[[-1]]  # Partly bad for strips due to a test
             },
             '2017F':{
                 "306422":[[-1]], # Partly bad for strips - 2 data readouts failed 
#                 "306423":[[-1]], # Partly bad for strips - 2 data readouts failed
#                 "306425":[[-1]], # Partly bad for strips - 2 data readouts failed
#                 "305440":[[-1]], # Partly bad for strips - 1 data readout failed
#                 "305441":[[-1]], # Partly bad for strips - 1 data readout failed
                 "305249":[[-1]], # Bad for pixels - half of disk failed 
                 "305250":[[-1]], # Bad for pixels - half of disk failed
#                 "305064":[[-1]], # Marginal for pixels - some readout failed
             },
            '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                {
                #"317479":[[-1]],
                "317480":[[-1]],
                "317481":[[-1]],
                "317482":[[-1]],
                #"319847":[[1,35]]
            }}


# The year and era being used
year = '2017'
era = 'B'

# Create a list of histograms to include
# Pair histograms to be combined on the same line
histnames = [
    ['NormalizedHitResiduals_TIB__Layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'NormalizedHitResiduals_TIB__Layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2',
     'NormalizedHitResiduals_TIB__Layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3' , 'NormalizedHitResiduals_TIB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4'],
    ['chargeInner_PXLayer_1', 'chargeOuter_PXLayer_1', 'adc_PXLayer_1'],
    ['chargeInner_PXLayer_2', 'chargeOuter_PXLayer_2', 'adc_PXLayer_2'],
    ['chargeInner_PXLayer_3', 'chargeOuter_PXLayer_3', 'adc_PXLayer_3'],
    ['chargeInner_PXLayer_4', 'chargeOuter_PXLayer_4', 'adc_PXLayer_4'],
    ['charge_PXDisk_+1', 'adc_PXDisk_+1'],
    ['charge_PXDisk_-1', 'adc_PXDisk_-1'],
    ['charge_PXDisk_+2', 'adc_PXDisk_+2'],
    ['charge_PXDisk_-2', 'adc_PXDisk_-2'],
    ['charge_PXDisk_+3', 'adc_PXDisk_+3'],
    ['charge_PXDisk_-3', 'adc_PXDisk_-3'],
    ['NormalizedHitResiduals_TOB__Layer__1', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__1', 'NormalizedHitResiduals_TOB__Layer__2', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__2',
     'NormalizedHitResiduals_TOB__Layer__3', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__3' , 'NormalizedHitResiduals_TOB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__4']
]

# Read new data or use previously saved data & should data be saved
readnew = True
save = False

In [16]:
### Define Run Properties - Combined Autoencoder
# in this cell all major run properties are going to be set,

# Set whether to train globally or locally or in a development/testing mode
training_mode = 'development'

# Selects whether to create a new histstruct or use a saved one
readnew = True

# Select whether to save a new histstruct
save = False

In [17]:
### Model Controls and Selection - 1D Autoencoder

plotNames = 'Test'
name = plotNames+'plots'

# Choose whether to train a new model or load one
trainnew = True
savemodel = True
modelname = plotNames

# Evaluate models seperately, as an ensemble, both, or neither
individualEval = True
ensembleEval = False

In [18]:
### Define Training Mode Parameters - Combined Autoencoder
if training_mode == 'global':
    runsls_training = None # use none to not add a mask for training (can e.g. use DCS-bit on mask)
    runsls_good = None # use none to not add a mask for good runs (can e.g. use averages of training set)
    runsls_bad = badrunsls[year] # predefined bad runs
    print('selected runs/lumisections for training: all')
    
elif training_mode == 'local':
    # train locally on a small set of runs
    # - either on n runs preceding a chosen application run,
    # - or on the run associated as reference to the chosen application run.
    # - this only works for a single era
    
    available_runs = dfu.get_runs( dfu.select_dcson( csvu.read_csv('../data/DF'+year+era+'_'+histnames[0][0]+'.csv') ) )
    # Cherry picked really bad run
    run_application = 299316
    #run_application = 299317
    run_application_index = available_runs.index(run_application)
    # select training set
    usereference = False
    if usereference:
        run_reference = rru.get_reference_run( run_application, jsonfile='../utils/json_allRunsRefRuns.json' )
        if run_reference<0:
            raise Exception('no valid reference run has been defined for run {}'.format(run_application))
        runsls_training = jsonu.tuplelist_to_jsondict([(run_reference,[-1])])
    else:
        ntraining = 5
        offset = 0 # normal case: offset = 0 (just use 5 previous runs)
        
        # Selects the 5 previous runs for training
        runsls_training = jsonu.tuplelist_to_jsondict([(el,[-1]) for el in available_runs[run_application_index-ntraining-offset:run_application_index-offset]])
    #runsls_bad = badrunsls[year]
    #runsls_good = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_bad = jsonu.tuplelist_to_jsondict([(run_application,[-1])])
    runsls_good = runsls_training
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)
        
elif training_mode == 'development':
    # train on a user-defined subset of runs
    
    # Select runs to be used in training from the user-defined list
    runsls_training = trainrunsls[year + era]
    # Select bad runs to test on in the user-defined list
    runsls_bad = badrunsls[year + era]
    # Select good runs to test on in the user-defined list
    runsls_good = goodrunsls[year + era]
    
    print('selected runs/lumisections for training: ')
    print(runsls_training)
    print('selected runs/lumisections as good test set:')
    print(runsls_good)
    print('selected runs/lumisections as bad test set:')
    print(runsls_bad)

selected runs/lumisections for training: 
{'297175': [[-1]], '297620': [[-1]], '297659': [[-1]], '297670': [[-1]], '299065': [[-1]], '299067': [[-1]], '299096': [[-1]], '299149': [[-1]]}
selected runs/lumisections as good test set:
{'297175': [[-1]], '297659': [[-1]], '297670': [[-1]], '297674': [[-1]], '297722': [[-1]], '299065': [[-1]], '299067': [[-1]], '299185': [[-1]], '299327': [[-1]], '299480': [[-1]]}
selected runs/lumisections as bad test set:
{'299316': [[-1]], '299317': [[-1]], '299318': [[-1]], '299324': [[-1]], '299326': [[-1]], '297047': [[-1]], '297169': [[-1]], '297664': [[-1]]}


In [20]:
### Data Import

# Create a new HistStruct from the data
if readnew:
    # Initializations
    dloader = DataLoader.DataLoader()
    histstruct = SubHistStruct.SubHistStruct()
    histstruct.reset_histlist(histnames)
    
    # Unpack histnames and add every histogram individually
    for histnamegroup in histnames:
        for histname in histnamegroup:
            print('Adding {}...'.format(histname))
            
            # Bring the histograms into memory from storage for later use
            filename = datadir + year + era + '/DF' + year + era + '_' + histname + '.csv'
            df = dloader.get_dataframe_from_file( filename )
            
            # In case of local training, we can remove most of the histograms
            if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
                runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
                df = dfu.select_runsls( df, runsls_total )
                
            # Store the data in the histstruct object managing this whole thing
            histstruct.add_dataframe( df, rebinningfactor = 3 )
        
    print('Found {} histograms'.format(len(histstruct.runnbs)))

# Load a previously saved HistStruct
else:
    # Load histstruct from storage
    histstruct = HistStruct.HistStruct.load('test.pk1')
    
    # Count of bad runs, presumably for later use
    nbadruns = len([name for name in list(histstruct.masks.keys()) if 'bad' in name])
    
if userfriendly:
    print('Created a histstruct with the following properties:')
    print('- number of histogram types: {}'.format(len(histstruct.histnames)))
    print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))

Adding NormalizedHitResiduals_TIB__Layer__1...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__1...
Adding NormalizedHitResiduals_TIB__Layer__2...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__2...
Adding NormalizedHitResiduals_TIB__Layer__3...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__3...
Adding NormalizedHitResiduals_TIB__Layer__4...
Adding Summary_ClusterStoNCorr__OnTrack__TIB__layer__4...
Adding chargeInner_PXLayer_1...
Adding chargeOuter_PXLayer_1...
Adding adc_PXLayer_1...
WARNING in hist_utils.py / rebinhists: no rebinning performed since no suitable reduction factor was given. The rebinning factor (3) is not a divisor of the number of bins (34)
Adding chargeInner_PXLayer_2...
Adding chargeOuter_PXLayer_2...
Adding adc_PXLayer_2...
WARNING in hist_utils.py / rebinhists: no rebinning performed since no suitable reduction factor was given. The rebinning factor (3) is not a divisor of the number of bins (34)
Adding chargeInner_PXLayer_3...
Adding chargeOuter

In [21]:
### Add Masks to Data

if readnew:
    histstruct.add_dcsonjson_mask( 'dcson' )
    histstruct.add_goldenjson_mask('golden' )
    histstruct.add_highstat_mask( 'highstat' )
    histstruct.add_stat_mask( 'lowstat', max_entries_to_bins_ratio=100 )
    if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
    if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
        
    # Distinguishing bad runs
    nbadruns = 0
    if runsls_bad is not None:
        print(runsls_bad)
        histstruct.add_json_mask( 'bad', runsls_bad )
        
        # Special case for bad runs: add a mask per run (different bad runs have different characteristics)
        nbadruns = len(runsls_bad.keys())
        for i,badrun in enumerate(runsls_bad.keys()):
            histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
            
    if save:
        histstruct.save('test.pk1')
if userfriendly: print('Assigned masks: {}'.format(list(histstruct.masks.keys())))

{'299316': [[-1]], '299317': [[-1]], '299318': [[-1]], '299324': [[-1]], '299326': [[-1]], '297047': [[-1]], '297169': [[-1]], '297664': [[-1]]}
Assigned masks: ['dcson', 'golden', 'highstat', 'lowstat', 'training', 'good', 'bad', 'bad0', 'bad1', 'bad2', 'bad3', 'bad4', 'bad5', 'bad6', 'bad7']


In [22]:
### Plotting the input data for analysis

if((training_mode=='local' or training_mode == 'development') and createPlots):

    # training and application runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','training'],['dcson','highstat','good']],
                                labellist = ['training','testing'],
                                colorlist = ['blue','green']
                              )
    
    # application run and bad test runs
    histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad']],
                                labellist = ['good','bad'],
                                colorlist = ['green','red']
                              )
    
elif( training_mode=='global' and createPlots):
    
    # bad test runs
    for i in [0,1,2,3,4,5,6]:
        histstruct.plot_histograms( masknames=[['dcson','highstat','good'],['dcson','highstat','bad{}'.format(i)]],
                                labellist = ['typical good histograms','bad'],
                                colorlist = ['blue','red'],
                                transparencylist = [0.01,1.]
                                    )

In [29]:
def define_concatamash_autoencoder(histstruct):
    
    histslist = []
    vallist = []
    autoencoders = []
    if trainnew:
        for i,histnamegroup in enumerate(histnames):
            
            train_normhist = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'training']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
            X_train, X_val = train_test_split(train_normhist, test_size=0.4, random_state=42)
            
            if userfriendly:
                print('\nNow Defining model {}/'.format(i + 1) 
                      + str(len(histnames)))
                print(' - Size of training set: {}'.format(X_train.shape))
            
            # Half the total bin count
            arch = 51 * len(histnamegroup)
            
            ## Model parameters
            print(X_train.shape)
            
            input_dim = X_train.shape[2] #num of predictor variables
            Input_layers=[Input(shape=input_dim) for i in range((X_train.shape[1]))]
            
            # Defining layers
            conc_layer = Concatenate()(Input_layers)
            encoder = Dense(arch * 2, activation="tanh")(conc_layer)
            #encoder = Dense(128, activation='relu')(encoder)
            #
            #encoder = Dense(32, activation='relu')(encoder)
            
            decoder = Dense(arch, activation="relu")(encoder)
            #decoder = Dense(256, activation="tanh")(decoder)
            
            Output_layers=[Dense(input_dim, activation="tanh")(decoder) for i in range(X_train.shape[1])]

            autoencoder = Model(inputs=Input_layers, outputs=Output_layers)
            autoencoder.summary()
            autoencoders.append(autoencoder)
            
            histslist.append(X_train)
            vallist.append(X_val)
     
    # Return the histograms stored 2-Dimensionally and the autoencoders corresponding
    return(histslist, vallist, autoencoders, train_normhist)

In [30]:
(histslist, vallist, autoencoders, train_normhist) = define_concatamash_autoencoder(histstruct)


Now Defining model 1/12
 - Size of training set: (1006, 8, 34)
(1006, 8, 34)
Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_65 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_66 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_67 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_68 (InputLayer)           [(None, 34)]         0                                            
_____________

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_79 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_80 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_81 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
concatenate_18 (Concatenate)    (None, 102)          0           input_79[0][0]                   
                                                                 input_80[0][0]            

Model: "model_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_93 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
input_94 (InputLayer)           [(None, 34)]         0                                            
__________________________________________________________________________________________________
concatenate_24 (Concatenate)    (None, 68)           0           input_93[0][0]                   
                                                                 input_94[0][0]                   
__________________________________________________________________________________________________
dense_143 (Dense)               (None, 204)          14076       concatenate_24[0][0]      

In [36]:
### Trains a combined autoencoder for every merge set
def train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders):
    
    # Iterate through the training data to train corresponding autoencoders
    for i in range(len(histslist)):
        
        if userfriendly: print('\nNow training model {}/'.format(i + 1) + str(len(histslist)))
        
        # Set variables to temporary values for better transparency
        X_train = histslist[i]
        X_val = vallist[i]
        autoencoder = autoencoders[i]
        
        
        ## Model parameters
        nb_epoch = 500
        batch_size = 50
        
        #checkpoint_filepath = 'checkpoint'
        #model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        #   filepath=checkpoint_filepath,
        #   save_weights_only=False,
        #   verbose=1,
        #   save_best_only=True,
        #   monitor='val_loss',
        #   mode='min')
        
        # Tell the model when to stop
        earlystop = EarlyStopping(monitor='val_loss',
            min_delta=1e-7,
            patience=20,
            verbose=1,
            mode='auto',
            baseline=None,
            restore_best_weights=True,
        )
        lr =0.001
        opt = keras.optimizers.Adam(learning_rate=lr)
        
        autoencoder.compile(loss='mse',
                            optimizer=opt)
        
        ## Train autoencoder
        train = autoencoder.fit(x=[X_train[:,i] for i in range(X_train.shape[1])],
                                y=[X_train[:,i] for i in range(X_train.shape[1])],
                            epochs=nb_epoch,
                            batch_size=batch_size,
                            shuffle=True,
                            validation_data=([X_val[:,i] for i in range(X_val.shape[1])], [X_val[:,i] for i in range(X_val.shape[1])]),
                            verbose=1,
                            callbacks= [earlystop],    
                            )
        
        # Create a plot of the model
        
        tf.keras.utils.plot_model(
            autoencoder,
            to_file="models/modelConcatamash{}.png".format(i),
            show_shapes=True,
            show_dtype=False,
            show_layer_names=False,
            rankdir="TB")
        
        # Save classifier for evaluation
        classifier = AutoEncoder.AutoEncoder(model=autoencoder)
        histstruct.add_classifier(histnames[i][0], classifier)        

In [37]:
start = time.perf_counter()
train_concatamash_autoencoder(histstruct, histslist, vallist, autoencoders)
stop = time.perf_counter()
print(stop - start)


Now training model 1/12
Epoch 1/500
21/21 [==============================] - 2s 23ms/step - loss: 7.7848e-04 - dense_99_loss: 1.0051e-04 - dense_100_loss: 9.9550e-05 - dense_101_loss: 1.1147e-04 - dense_102_loss: 9.1561e-05 - dense_103_loss: 9.1641e-05 - dense_104_loss: 9.7742e-05 - dense_105_loss: 9.0888e-05 - dense_106_loss: 9.5123e-05 - val_loss: 1.6482e-04 - val_dense_99_loss: 2.0207e-05 - val_dense_100_loss: 2.6196e-05 - val_dense_101_loss: 1.4375e-05 - val_dense_102_loss: 3.4918e-05 - val_dense_103_loss: 1.9823e-05 - val_dense_104_loss: 2.0088e-05 - val_dense_105_loss: 1.6580e-05 - val_dense_106_loss: 1.2638e-05
Epoch 2/500
21/21 [==============================] - 0s 5ms/step - loss: 6.9044e-05 - dense_99_loss: 7.7916e-06 - dense_100_loss: 1.0513e-05 - dense_101_loss: 7.0906e-06 - dense_102_loss: 9.6417e-06 - dense_103_loss: 8.6378e-06 - dense_104_loss: 9.6944e-06 - dense_105_loss: 7.6311e-06 - dense_106_loss: 8.0435e-06 - val_loss: 2.3462e-05 - val_dense_99_loss: 2.2142e-06 - v

Epoch 15/500
21/21 [==============================] - 0s 5ms/step - loss: 1.4429e-05 - dense_99_loss: 1.2891e-06 - dense_100_loss: 3.4864e-06 - dense_101_loss: 1.2843e-06 - dense_102_loss: 2.0087e-06 - dense_103_loss: 1.5287e-06 - dense_104_loss: 1.5921e-06 - dense_105_loss: 1.8101e-06 - dense_106_loss: 1.4292e-06 - val_loss: 1.1670e-05 - val_dense_99_loss: 8.9719e-07 - val_dense_100_loss: 2.7898e-06 - val_dense_101_loss: 8.7783e-07 - val_dense_102_loss: 1.7505e-06 - val_dense_103_loss: 1.4300e-06 - val_dense_104_loss: 1.2163e-06 - val_dense_105_loss: 1.4927e-06 - val_dense_106_loss: 1.2155e-06
Epoch 16/500
21/21 [==============================] - 0s 5ms/step - loss: 1.1740e-05 - dense_99_loss: 9.4107e-07 - dense_100_loss: 2.9911e-06 - dense_101_loss: 9.4649e-07 - dense_102_loss: 1.6938e-06 - dense_103_loss: 1.1912e-06 - dense_104_loss: 1.3013e-06 - dense_105_loss: 1.4819e-06 - dense_106_loss: 1.1931e-06 - val_loss: 1.1393e-05 - val_dense_99_loss: 9.6158e-07 - val_dense_100_loss: 2.734

Epoch 29/500
21/21 [==============================] - 0s 5ms/step - loss: 8.2876e-06 - dense_99_loss: 7.6137e-07 - dense_100_loss: 1.6161e-06 - dense_101_loss: 7.6106e-07 - dense_102_loss: 1.0143e-06 - dense_103_loss: 1.0233e-06 - dense_104_loss: 9.9106e-07 - dense_105_loss: 1.2393e-06 - dense_106_loss: 8.8115e-07 - val_loss: 9.6711e-06 - val_dense_99_loss: 7.9786e-07 - val_dense_100_loss: 1.7781e-06 - val_dense_101_loss: 9.4928e-07 - val_dense_102_loss: 1.4681e-06 - val_dense_103_loss: 1.2775e-06 - val_dense_104_loss: 1.1965e-06 - val_dense_105_loss: 1.2945e-06 - val_dense_106_loss: 9.0931e-07
Epoch 30/500
21/21 [==============================] - 0s 5ms/step - loss: 1.4921e-05 - dense_99_loss: 1.6458e-06 - dense_100_loss: 2.2010e-06 - dense_101_loss: 1.4546e-06 - dense_102_loss: 2.0351e-06 - dense_103_loss: 1.6696e-06 - dense_104_loss: 2.0287e-06 - dense_105_loss: 2.1600e-06 - dense_106_loss: 1.7267e-06 - val_loss: 8.2079e-06 - val_dense_99_loss: 8.5684e-07 - val_dense_100_loss: 1.411

Epoch 43/500
21/21 [==============================] - 0s 5ms/step - loss: 6.5285e-06 - dense_99_loss: 6.3593e-07 - dense_100_loss: 9.5514e-07 - dense_101_loss: 7.0073e-07 - dense_102_loss: 7.5714e-07 - dense_103_loss: 9.0745e-07 - dense_104_loss: 7.5244e-07 - dense_105_loss: 1.0416e-06 - dense_106_loss: 7.7810e-07 - val_loss: 7.5228e-06 - val_dense_99_loss: 7.3812e-07 - val_dense_100_loss: 1.2121e-06 - val_dense_101_loss: 8.8139e-07 - val_dense_102_loss: 8.6909e-07 - val_dense_103_loss: 9.9426e-07 - val_dense_104_loss: 8.7500e-07 - val_dense_105_loss: 1.0875e-06 - val_dense_106_loss: 8.6537e-07
Epoch 44/500
21/21 [==============================] - 0s 5ms/step - loss: 6.9216e-06 - dense_99_loss: 6.7956e-07 - dense_100_loss: 9.8612e-07 - dense_101_loss: 6.7079e-07 - dense_102_loss: 8.3750e-07 - dense_103_loss: 9.8046e-07 - dense_104_loss: 8.1260e-07 - dense_105_loss: 1.1386e-06 - dense_106_loss: 8.1603e-07 - val_loss: 6.6575e-06 - val_dense_99_loss: 6.1222e-07 - val_dense_100_loss: 9.446

Epoch 57/500
21/21 [==============================] - 0s 5ms/step - loss: 8.3502e-06 - dense_99_loss: 9.4127e-07 - dense_100_loss: 9.3511e-07 - dense_101_loss: 9.6598e-07 - dense_102_loss: 9.0677e-07 - dense_103_loss: 1.2282e-06 - dense_104_loss: 9.7221e-07 - dense_105_loss: 1.2779e-06 - dense_106_loss: 1.1227e-06 - val_loss: 8.1327e-06 - val_dense_99_loss: 9.4699e-07 - val_dense_100_loss: 1.0615e-06 - val_dense_101_loss: 8.1340e-07 - val_dense_102_loss: 8.8668e-07 - val_dense_103_loss: 1.1340e-06 - val_dense_104_loss: 8.7728e-07 - val_dense_105_loss: 1.2422e-06 - val_dense_106_loss: 1.1706e-06
Epoch 58/500
21/21 [==============================] - 0s 5ms/step - loss: 6.3081e-06 - dense_99_loss: 7.3605e-07 - dense_100_loss: 7.4460e-07 - dense_101_loss: 7.0768e-07 - dense_102_loss: 6.2249e-07 - dense_103_loss: 9.7010e-07 - dense_104_loss: 6.8764e-07 - dense_105_loss: 1.0335e-06 - dense_106_loss: 8.0597e-07 - val_loss: 7.4994e-06 - val_dense_99_loss: 7.2468e-07 - val_dense_100_loss: 1.067

...

Epoch 22/500
21/21 [==============================] - 0s 5ms/step - loss: 1.3864e-05 - dense_153_loss: 1.2364e-06 - dense_154_loss: 2.4424e-06 - dense_155_loss: 1.1093e-06 - dense_156_loss: 2.0957e-06 - dense_157_loss: 1.4856e-06 - dense_158_loss: 1.8616e-06 - dense_159_loss: 1.7509e-06 - dense_160_loss: 1.8821e-06 - val_loss: 1.2782e-05 - val_dense_153_loss: 1.0181e-06 - val_dense_154_loss: 2.4388e-06 - val_dense_155_loss: 9.7703e-07 - val_dense_156_loss: 1.8788e-06 - val_dense_157_loss: 1.4540e-06 - val_dense_158_loss: 1.5547e-06 - val_dense_159_loss: 1.7173e-06 - val_dense_160_loss: 1.7434e-06
Epoch 23/500
21/21 [==============================] - 0s 5ms/step - loss: 1.7459e-05 - dense_153_loss: 1.6127e-06 - dense_154_loss: 3.0048e-06 - dense_155_loss: 1.5338e-06 - dense_156_loss: 2.4318e-06 - dense_157_loss: 1.9438e-06 - dense_158_loss: 2.0754e-06 - dense_159_loss: 2.4202e-06 - dense_160_loss: 2.4368e-06 - val_loss: 1.3403e-05 - val_dense_153_loss: 1.3306e-06 - val_dense_154_loss: 2

Epoch 36/500
21/21 [==============================] - 0s 5ms/step - loss: 1.2293e-05 - dense_153_loss: 1.2750e-06 - dense_154_loss: 1.5682e-06 - dense_155_loss: 1.2082e-06 - dense_156_loss: 1.4047e-06 - dense_157_loss: 1.6250e-06 - dense_158_loss: 1.6214e-06 - dense_159_loss: 1.8072e-06 - dense_160_loss: 1.7831e-06 - val_loss: 1.2257e-05 - val_dense_153_loss: 1.0155e-06 - val_dense_154_loss: 1.7590e-06 - val_dense_155_loss: 1.1699e-06 - val_dense_156_loss: 1.6709e-06 - val_dense_157_loss: 1.7523e-06 - val_dense_158_loss: 1.4638e-06 - val_dense_159_loss: 1.6346e-06 - val_dense_160_loss: 1.7911e-06
Epoch 37/500
21/21 [==============================] - 0s 5ms/step - loss: 1.0589e-05 - dense_153_loss: 9.7728e-07 - dense_154_loss: 1.4725e-06 - dense_155_loss: 1.0473e-06 - dense_156_loss: 1.3000e-06 - dense_157_loss: 1.4915e-06 - dense_158_loss: 1.2898e-06 - dense_159_loss: 1.5611e-06 - dense_160_loss: 1.4500e-06 - val_loss: 1.2762e-05 - val_dense_153_loss: 1.1946e-06 - val_dense_154_loss: 1

Epoch 50/500
21/21 [==============================] - 0s 5ms/step - loss: 7.7957e-06 - dense_153_loss: 7.2648e-07 - dense_154_loss: 8.6762e-07 - dense_155_loss: 7.2584e-07 - dense_156_loss: 9.0529e-07 - dense_157_loss: 1.1586e-06 - dense_158_loss: 1.0319e-06 - dense_159_loss: 1.2466e-06 - dense_160_loss: 1.1335e-06 - val_loss: 8.0889e-06 - val_dense_153_loss: 7.4120e-07 - val_dense_154_loss: 9.6018e-07 - val_dense_155_loss: 8.8808e-07 - val_dense_156_loss: 9.5624e-07 - val_dense_157_loss: 1.3002e-06 - val_dense_158_loss: 1.0065e-06 - val_dense_159_loss: 1.1987e-06 - val_dense_160_loss: 1.0377e-06
Epoch 51/500
21/21 [==============================] - 0s 5ms/step - loss: 8.1796e-06 - dense_153_loss: 7.9883e-07 - dense_154_loss: 8.8382e-07 - dense_155_loss: 8.2780e-07 - dense_156_loss: 9.4720e-07 - dense_157_loss: 1.2247e-06 - dense_158_loss: 1.0932e-06 - dense_159_loss: 1.2846e-06 - dense_160_loss: 1.1194e-06 - val_loss: 7.4757e-06 - val_dense_153_loss: 6.8657e-07 - val_dense_154_loss: 8

Epoch 64/500
21/21 [==============================] - 0s 5ms/step - loss: 8.0456e-06 - dense_153_loss: 8.3093e-07 - dense_154_loss: 8.2144e-07 - dense_155_loss: 8.6020e-07 - dense_156_loss: 8.8433e-07 - dense_157_loss: 1.2501e-06 - dense_158_loss: 1.0801e-06 - dense_159_loss: 1.2347e-06 - dense_160_loss: 1.0838e-06 - val_loss: 8.3389e-06 - val_dense_153_loss: 7.2970e-07 - val_dense_154_loss: 9.0584e-07 - val_dense_155_loss: 8.2763e-07 - val_dense_156_loss: 8.7502e-07 - val_dense_157_loss: 1.2790e-06 - val_dense_158_loss: 1.1489e-06 - val_dense_159_loss: 1.5080e-06 - val_dense_160_loss: 1.0649e-06
Epoch 65/500
21/21 [==============================] - 0s 5ms/step - loss: 1.1136e-05 - dense_153_loss: 1.1102e-06 - dense_154_loss: 1.3350e-06 - dense_155_loss: 1.0539e-06 - dense_156_loss: 1.2734e-06 - dense_157_loss: 1.6154e-06 - dense_158_loss: 1.5338e-06 - dense_159_loss: 1.7131e-06 - dense_160_loss: 1.5016e-06 - val_loss: 1.5384e-05 - val_dense_153_loss: 2.1000e-06 - val_dense_154_loss: 1

Epoch 78/500
21/21 [==============================] - 0s 5ms/step - loss: 7.1846e-06 - dense_153_loss: 7.2718e-07 - dense_154_loss: 6.9091e-07 - dense_155_loss: 7.3777e-07 - dense_156_loss: 8.5146e-07 - dense_157_loss: 1.0732e-06 - dense_158_loss: 9.3298e-07 - dense_159_loss: 1.2158e-06 - dense_160_loss: 9.5527e-07 - val_loss: 9.3527e-06 - val_dense_153_loss: 9.2990e-07 - val_dense_154_loss: 1.0491e-06 - val_dense_155_loss: 8.4933e-07 - val_dense_156_loss: 1.0762e-06 - val_dense_157_loss: 1.4510e-06 - val_dense_158_loss: 1.2466e-06 - val_dense_159_loss: 1.4346e-06 - val_dense_160_loss: 1.3161e-06
Epoch 79/500
21/21 [==============================] - 0s 5ms/step - loss: 7.0656e-06 - dense_153_loss: 7.0797e-07 - dense_154_loss: 6.9955e-07 - dense_155_loss: 7.4839e-07 - dense_156_loss: 7.3146e-07 - dense_157_loss: 1.1002e-06 - dense_158_loss: 8.9761e-07 - dense_159_loss: 1.1844e-06 - dense_160_loss: 9.9600e-07 - val_loss: 8.0123e-06 - val_dense_153_loss: 7.9498e-07 - val_dense_154_loss: 9

In [ ]:
### Evaluate the Models for WP definition
def evaluate_models_train(histstruct):
    
    for histgroup in histnames:
        print('evaluating model for '+histgroup[0])
        print(histstruct.evaluate_classifier(histgroup)[0].shape)
    
    # get mse for training set
    if 'training' in histstruct.masks.keys(): masknames = ['dcson','highstat', 'training']
    else: masknames = ['dcson','highstat']
    mse_train = histstruct.get_scores_array( masknames=masknames )
    print('Found mse array for training set of following shape: {}'.format(mse_train.shape))
    
    # get mse for good set
    if 'good' in histstruct.masks.keys():
        mse_good = []
        for histname in histstruct.histnames:
            mse_good.append(histstruct.get_scores( histname=histname, masknames=['dcson','highstat','good'] ))
    else:
        mse_good = []
        for histname in histstruct.histnames:
            hists_good = hu.averagehists( histstruct.get_histograms( histname=histname, masknames=['dcson','highstat']), 1000 )
            thismse = histstruct.classifiers[histname].evaluate( hists_good )
            mse_good.append( thismse )
            print(run)
    mse_good = np.array(mse_good)
    mse_good = np.transpose(mse_good)
    print('Found mse array for good set of following shape: {}'.format(mse_good.shape))
    
    # get mse for bad sets
    mse_bad = []
    for i in range(nbadruns):
        mse_bad.append( histstruct.get_scores_array( masknames=['dcson','highstat','bad{}'.format(i)] ) )
        print('Found mse array for bad set of following shape: {}'.format(mse_bad[i].shape))
        
    return [mse_train, mse_good, mse_bad]

In [ ]:
(mse_train, mse_good_eval, mse_bad_eval) = evaluate_models_train(histstruct)

In [ ]:
### Plots and Distribution Analysis
def fit_mse_distribution(histstruct, mse_train):
    dimslist = []
    fitfunclist = []
    
    
    nhisttypes = len(histstruct.histnames)
    for i in range(0,nhisttypes-1):
        for j in range(i+1,nhisttypes):
            dimslist.append((i, j))
    
    plt.close('all')
    (npoints,ndims) = mse_train.shape
    
    
    # settings for GaussianKdeFitter
    scott_bw = npoints**(-1./(ndims+4))
    bw_method = 20*scott_bw
    # settings for HyperRectangleFitter
    quantiles = ([0.00062,0.0006,0.00015,0.00015,
                 0.0003,0.0003,0.00053,0.00065])
    
    
    #for dims in dimslist:
    #    thismse = mse_train[:,dims]
    #    if training_mode=='global': 
    #        fitfunc = SeminormalFitter.SeminormalFitter(thismse)
    #        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(thismse, 
    #        #                                                    [quantiles[dims[0]],quantiles[dims[1]]],
    #        #                                                    'up')
    #    else: fitfunc = GaussianKdeFitter.GaussianKdeFitter(thismse,bw_method=bw_method)
    #    #pu.plot_fit_2d(thismse, fitfunc=fitfunc, logprob=True, clipprob=True,
    #    #                onlycontour=False, xlims=30, ylims=30, 
    #    #                onlypositive=True, transparency=0.5,
    #    #                xaxtitle=histstruct.histnames[dims[0]], 
    #    #                yaxtitle=histstruct.histnames[dims[1]],
    #    #                title='density fit of lumisection MSE')
    #    ##plt.close('all') # release plot memory
    #    fitfunclist.append(fitfunc)
    # 
    #    
    if training_mode=='global': 
        fitfunc = SeminormalFitter.SeminormalFitter(mse_train)
        #fitfunc = HyperRectangleFitter.HyperRectangleFitter(mse_train, quantiles, 'up')
    else: 
        fitfunc = GaussianKdeFitter.GaussianKdeFitter()
        fitfunc.fit(mse_train,bw_method=bw_method)
    
    return fitfunc

In [ ]:
fitfunc = fit_mse_distribution(histstruct, mse_train)

In [ ]:
### Prepare MSEs for Working Point Definition
def mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc):
    
    # Get the minimum log probability of histograms in good set
    print('--- good lumesections ---')
    logprob_good = np.log(fitfunc.pdf(mse_good_eval))
    print('length of log prob array: '+str(len(logprob_good)))
    print('minimum of log prob: '+str(np.min(logprob_good)))
    #print(sorted(logprob_good))
    
    print('--- bad lumisections ---')
    logprob_bad_parts = [np.log(fitfunc.pdf(mse_bad_eval[j])) for j in range(len(mse_bad_eval))]
    #for lp in logprob_bad_parts: print(str(sorted(lp))+'\n\n')
    logprob_bad = np.concatenate(tuple(logprob_bad_parts))
    
    print('length of log prob array: '+str(len(logprob_bad)))
    print('maximum of log prob: '+str(np.max(logprob_bad)))
    #print(sorted(logprob_good))
    #print(sorted(logprob_bad))
    #print(logprob_bad)
    
    return [logprob_good, logprob_bad]

In [ ]:
(logprob_good, logprob_bad) = mse_analysis(histstruct, mse_good_eval, mse_bad_eval, fitfunc)

In [ ]:
def evaluate_autoencoder_individual(histstruct):
    
    msewps = []
    for histname in histstruct.histnames:
        
        # Get histograms from histstruct
        X_test_good = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'good']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
        
        X_test_bad = X_test_bad = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'bad']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
        
        # Get each model from the histstruct
        autoencoder = histstruct.get_classifier(histname)
        
        # Getting evaluation criteria
        prediction_test_good = autoencoder.reconstruct(X_test_good)
        mse_test_good = aeu.mseTopNRaw(X_test_good, prediction_test_good, n=10 )
        prediction_test_bad = autoencoder.reconstruct(X_test_bad)
        mse_test_bad = aeu.mseTopNRaw(X_test_bad, prediction_test_bad, n=10 )
        
        if userfriendly:
            print('Average MSE on good set: ' + str(np.mean(mse_test_good)))
            print('Average MSE on bad set: ' + str(np.mean(mse_test_bad)))
        
        if createPlots:
            # Number of plots of each type to generate per model (so nplot * 2 * len(model))
            nplot = 3
            
            # Good examples
            print('Examples of good histograms and reconstruction:')
            randint = np.random.choice(np.arange(len(X_test_good)),size=nplot,replace=False)
            for i in randint: 
                histlist = [X_test_good[int(i),:],prediction_test_good[int(i),:]]
                labellist = ['data','reconstruction']
                colorlist = ['black','blue']
                pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                plt.show()
            
            # Bad examples
            print('Examples of bad histograms and reconstruction:')
            randint = np.random.choice(np.arange(len(X_test_bad)),size=nplot,replace=False)
            for i in randint:
                histlist = [X_test_bad[int(i),:],prediction_test_bad[int(i),:]]
                labellist = ['data','reconstruction']
                colorlist = ['black','blue']
                pu.plot_hists(histlist,colorlist=colorlist,labellist=labellist)
                plt.show()
        
        # Attaching the bad histograms as a new set of rows under the good histograms
        validation_data = np.vstack((X_test_good, X_test_bad))
        validation_preds = np.vstack((prediction_test_good, prediction_test_bad))
        # Creating labels to differentiate the data when we go to compare predictions
        #     with actual label
        labels = np.hstack((np.zeros(len(X_test_good)), np.ones(len(X_test_bad))))
        
        # Pick a working point to see 
        msewp = 0.5*(np.mean(mse_test_bad) - np.mean(mse_test_good))
        print("Selected working point: " + str(msewp))
        
        # Get data to pick a good working point for future evaluation
        scores = aeu.mseTop10Raw(validation_data, validation_preds)
        nsig = np.sum(labels)
        nback = np.sum(1-labels)
        
        # Get some metrics for the user
        tp = np.sum(np.where((labels==1) & (scores>msewp),1,0))/nsig
        fp = np.sum(np.where((labels==0) & (scores>msewp),1,0))/nback
        tn = 1-fp
        fn = 1-tp
        
        accuracy = (tp + tn) / (tp + fp + tn + fn)
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = (2*precision*recall) / (precision + recall)
        
        if userfriendly:
            print(accuracy)
            print(precision)
            print(recall)
            print(f1)
        
    return msewps

In [ ]:
def evaluate_autoencoders_combined(logprob_good, logprob_bad, biasFactor):
    labels_good = np.zeros(len(logprob_good)) # background: label = 0
    labels_bad = np.ones(len(logprob_bad)) # signal: label = 1
    
    badMin = min(np.where(logprob_bad != -np.inf, logprob_bad, np.inf))
    goodMax = max(np.where(logprob_good != np.inf, logprob_good, -np.inf))
    
    # Getting rid of infinities
    logprob_good[logprob_good > 500] = goodMax
    logprob_bad[logprob_bad < 0] = badMin
    # These only take effect if a histogram is grossly misclassified
    logprob_good[logprob_good < badMin] = badMin
    logprob_bad[logprob_bad > goodMax] = goodMax
    
    labels = np.concatenate(tuple([labels_good,labels_bad]))
    scores = np.concatenate(tuple([-logprob_good,-logprob_bad]))
    scores = aeu.clip_scores( scores )
    
    pu.plot_score_dist(scores, labels, siglabel='anomalous', sigcolor='r', 
                       bcklabel='good', bckcolor='g', 
                       nbins=200, normalize=True,
                       xaxtitle='negative logarithmic probability',
                       yaxtitle='number of lumisections (normalized)')
      
    # Plot ROC curve for analysis
    auc = aeu.get_roc(scores, labels, mode='geom', doprint=False)
    
    # Setting a threshold, below this working point defines anomalous data
    # Average is biased towards better recall per user specifications
    logprob_threshold = (1/(biasFactor + 1)) * (biasFactor*np.mean(logprob_good) + np.mean(logprob_bad))
    # Or set manual
    # logprob_threshold = 424
    (_, _, _, tp, fp, tn, fn) = aeu.get_confusion_matrix(scores,labels,-logprob_threshold)
    print('Selected logprob threshold of ' + str(logprob_threshold))
    
    # Get metrics for analysis
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_measure = (1 + biasFactor * biasFactor) * ((precision * recall) / ((biasFactor * biasFactor * precision) + recall)) 
    
    print('Accuracy: ' + str(accuracy))
    print('Precision: ' + str(precision))
    print('Recall: ' + str(recall))
    print('F-Measure: ' + str(f_measure))
    
    return logprob_threshold

In [ ]:
logprob_threshold = evaluate_autoencoders_combined(logprob_good, logprob_bad, biasFactor)